# TSA Chapter 2: Case Study: Sunspot Data ARMA Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch2/TSA_ch2_case_study/TSA_ch2_case_study.ipynb)

This notebook demonstrates:
- Complete ARMA modeling case study on sunspot data
- ACF/PACF analysis, AR(2) fitting, residual diagnostics, and 20-year forecasting

In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from matplotlib.patches import Polygon
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    """Save chart as PDF and PNG."""
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
# Load sunspot data
from statsmodels.datasets import sunspots
data = sunspots.load_pandas().data
y = data['SUNACTIVITY'].values

# Create comprehensive figure

In [ ]:
fig = plt.figure(figsize=(14, 10))

# Plot 1: Raw data
ax1 = fig.add_subplot(2, 2, 1)
ax1.plot(y, 'b-', linewidth=0.8)
ax1.set_title('Sunspot Numbers (1700-2008)', fontsize=11)
ax1.set_xlabel('Year')
ax1.set_ylabel('Sunspot Count')
ax1.grid(True, alpha=0.3)

# Plot 2: ACF
ax2 = fig.add_subplot(2, 2, 2)
plot_acf(y, lags=30, ax=ax2)
ax2.set_title('ACF - Suggests AR or ARMA', fontsize=11)

# Fit AR(2) model
model = ARIMA(y, order=(2, 0, 0)).fit()

# Plot 3: Residual ACF
ax3 = fig.add_subplot(2, 2, 3)
plot_acf(model.resid, lags=20, ax=ax3)
ax3.set_title('Residual ACF (AR(2) fit)', fontsize=11)

# Plot 4: Forecast
ax4 = fig.add_subplot(2, 2, 4)
forecast = model.get_forecast(steps=20)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

ax4.plot(y[-50:], 'b-', linewidth=1, label='Observed')
ax4.plot(range(len(y[-50:]), len(y[-50:]) + 20), forecast_mean, 'r-', linewidth=2, label='Forecast')
ax4.fill_between(range(len(y[-50:]), len(y[-50:]) + 20),
                  forecast_ci.iloc[:, 0] if hasattr(forecast_ci, "iloc") else forecast_ci[:, 0],
                  forecast_ci.iloc[:, 1] if hasattr(forecast_ci, "iloc") else forecast_ci[:, 1],
                  color='red', alpha=0.2, label='95% CI')
ax4.set_title('20-Year Forecast', fontsize=11)
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
save_chart(fig, 'ch2_case_study')
plt.show()

# Model summary
print("AR(2) Model Summary:")
print(f"  phi1 = {model.params[1]:.4f}")
print(f"  phi2 = {model.params[2]:.4f}")
print(f"  AIC = {model.aic:.2f}")
print(f"  BIC = {model.bic:.2f}")

# Ljung-Box test
lb_test = acorr_ljungbox(model.resid, lags=[10], return_df=True)
print(f"  Ljung-Box(10) p-value = {lb_test['lb_pvalue'].values[0]:.4f}")